In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, Conv2D, MaxPool2D, Flatten, Input, BatchNormalization
from tensorflow.keras.models import  Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.activations import relu, sigmoid


Extract Import datasets for bald & not bald images

In [ ]:
filedr = "/Users/mepeeqeek/Python Projects/Bald Detect d Child"
filedr_bald = "/Users/mepeeqeek/Python Projects/Bald Detect d Child/Dataset/Train/Bald"
filedr_ntbald = "/Users/mepeeqeek/Python Projects/Bald Detect d Child/Dataset/Train/NotBald"

In [ ]:
import os
files_bald = os.listdir(filedr_bald)
files_ntbald = os.listdir(filedr_ntbald)

In [ ]:
images_bald = []
balds = []

for fle in files_bald:
    total = filedr_bald+'/' + fle
    image = cv2.imread(total)
    print(total)
    bald = int(0)
    balds.append(bald)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    images_bald.append(image)

In [ ]:
images_ntbald = []
baldnot = []
for fle_nb in files_ntbald:
    total_nb = filedr_ntbald+'/'+fle_nb
    image_nb = cv2.imread(total_nb)
    nt_bald = int(1)
    print(total_nb)
    baldnot.append(nt_bald)
    image_nb = cv2.cvtColor(image_nb,cv2.COLOR_BGR2RGB)
    images_ntbald.append(image_nb)

In [ ]:
len(balds)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(images_bald[67])

In [ ]:
print(balds[67])

In [ ]:
plt.imshow(images_bald[777])
print(balds[777])
cv2.destroyAllWindows()

In [ ]:
images_b_nt = []
images_b_nt= images_bald + images_ntbald
baldornot = []
baldornot  = balds + baldnot

In [ ]:
len(images_b_nt)

In [ ]:
plt.imshow(images_b_nt[7778])
print(baldornot[7778])

In [ ]:
images_f = np.array(images_b_nt)
baldornot_f = np.array(baldornot)

In [ ]:
images_f.shape


In [ ]:
len(baldornot)

In [ ]:
values, counts = np.unique(baldornot_f,return_counts=True)
print(counts)

In [ ]:
#Plotting the size of Bald and non bald person in the dataset

fig = plt.figure()
axs = fig.add_axes([0,0,1,1])
descript = ['Bald', 'Not Bald']
axs.bar(descript,counts)
plt.show()

save the list in the folder location

In [ ]:
np.save(filedr+'images.npy',images_f)
np.save(filedr+'detection.npy',baldornot_f)

splitting the dataset for test and train

In [ ]:
images_f_2 = images_f/255

In [ ]:
## importing necessary library for test train
from sklearn.model_selection import train_test_split

In [ ]:
X_test,X_train,Y_test,Y_train = train_test_split(images_f_2,baldornot_f,test_size=0.50)

In [ ]:
Y_train[0:5]

In [ ]:
X_train[0:5]

In [ ]:
## how to define Convolution

def Convolution(input_tensor,filters):
    x= Conv2D(filters=filters,kernel_size=(3,3),padding="same",strides=(1,1),kernel_regularizer=l2(0.001))(input_tensor)
    x= Dropout(0.1)(x)  #dropout- for determining the parameters we drop to model ie model can work fine during the "Training" but not during the "Test"
    x= Activation('relu')(x)  
    return x

In [ ]:
## Define the Model
#As the convolution is increased the baising gets increased and if it is decreased then accuracy can decrease. Hence, we will use "4- convolution layers" and if needed we will increase it further

def model(input_shape):
    inputs = Input((input_shape))
    conv_1 = Convolution(inputs,32)
    maxp_1 = MaxPool2D(pool_size=(2,2))(conv_1)
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPool2D(pool_size=(2,2))(conv_2)
    conv_3 = Convolution(maxp_2,128)
    maxp_3 = MaxPool2D(pool_size=(2,2))(conv_3)
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPool2D(pool_size=(2,2))(conv_4)
    conv_5 = Convolution(maxp_4,512)
    maxp_5 = MaxPool2D(pool_size=(2,2))(conv_5)
    flatten = Flatten()(maxp_5)
    dense_1 = Dense(64,activation='relu')(flatten)
    drop_1 = Dropout(0.2)(dense_1)
    output_1 = Dense(1,activation='sigmoid',name='bald_out')(drop_1)  
    model = tensorflow.keras.models.Model(inputs=[inputs],outputs=[output_1])
    model.compile(loss=["binary_crossentropy","mae"],optimizer='rmsprop',metrics=["accuracy"])
    return model 

In [ ]:
Model= model((80,80,3))

In [ ]:
Model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
file_s = 'Bald_Detection.h5'
checkpoint = ModelCheckpoint(file_s,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
Early_stop= tensorflow.keras.callbacks.EarlyStopping(patience=25,monitor='val_loss',restore_best_weights='True')
callback_list = [checkpoint,Early_stop]

In [ ]:
History = Model.fit(X_train,Y_train,batch_size=128,validation_data=(X_test,Y_test),epochs=25,callbacks=callback_list)

Evaluation of  model

In [ ]:
Model.evaluate(X_test,Y_test)

Model prediction

In [ ]:
pred = Model.predict(X_test)

plotting Epoch vs Loss (using history file is difficult to plot everytime we open this file)

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train','validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

In [ ]:
i=0
Pred_1=[]
while(i<len(pred)):
    Pred_1.append(int(np.round(pred[i])))
    i=i+1

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(Y_test,Pred_1)

In [ ]:
print(report)

In [ ]:
results = confusion_matrix(Y_test,Pred_1)

In [ ]:
import seaborn as sns
sns.heatmap(results,annot=True)

In [ ]:
def test_image(ind,images_f,images_f_2,Model):
    plt.imshow(images_f[ind])
    image_test = images_f
    print(image_test.shape)
    pred_1 = Model.predict(np.array([image_test]))
    bald_det_f = ['Not Bald','Bald']
    bald_det = int(np.round(pred_1))
    print("Predicted Person is  "+bald_det_f[bald_det])

In [ ]:
test_image(544,images_f,images_f_2,Model)